In [ ]:
!uv pip install selenium

In [ ]:
# Import libraries
import os
from openai import OpenAI
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import gradio as gr # love this!

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

class Website:
    def __init__(self, url):
        """
        Create this WebsiteSelenium object from the given URL using Selenium and BeautifulSoup.
        """
        self.url = url

        # Set up Selenium WebDriver with headless Chrome
        chrome_options = Options()
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Prevent detection
        chrome_options.add_argument("--disable-infobars")  # Disable "Chrome is being controlled" infobar
        
        service = Service()  # Use default ChromeDriver path
        driver = webdriver.Chrome(service=service, options=chrome_options)

        try:
            # Fetch the webpage
            driver.get(url)

            # Get the page source
            page_source = driver.page_source

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        finally:
            # Close the WebDriver
            driver.quit()

In [ ]:
# Initialise Ollama and model choices
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='hohoho')
OL_MODEL1 = "llama3.2:latest"
OL_MODEL2 = "gpt-oss:latest"
OL_MODEL3 = "phi3:latest"
OL_MODEL4 = "gemma3:270m"
OL_MODEL5 = "gemma3:4b"
OL_MODEL_LIST = [OL_MODEL1, OL_MODEL2, OL_MODEL3, OL_MODEL4, OL_MODEL5]

In [ ]:
# Define call_LLM function
def call_LLM(chosen_model, sys_prompt, user_prompt):
    response = ollama.chat.completions.create(
        model= chosen_model,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    result = response.choices[0].message.content
    return result

In [ ]:
# Define job_profile_summariser function
def job_profile_summariser(chosen_model, job_url):
    job_system_prompt = """
    You are a highly experienced recruitment director reviewing a job profile.
    You are able to decide what are the most relevant information in this job profile.
    Please list out the key aspects of the job profile for candidate assessment.
    Here is an example:
    Job title & summary: A title reflecting seniority and a brief overview of the role’s purpose.
    Key responsibilities: A detailed list of daily tasks, expectations, and major contributions.
    Required skills & competencies: technical abilities (e.g., software, tools) and behavioural attributes (e.g., leadership, communication).
    Qualifications & experience: mandatory education, certifications, and previous experience.
    """

    job_profile = Website(job_url)
    job_user_prompt = f"Here is the job profile:\n"
    job_user_prompt += job_profile.text
    job_user_prompt += "\nPlease create a succinct and coherent summary of this job profile."
    
    return call_LLM(
        chosen_model=chosen_model,
        sys_prompt= job_system_prompt,
        user_prompt= job_user_prompt
    )

In [ ]:
# Define candidate_profile_summariser function
def candidate_profile_summariser(chosen_model, profile_url):
    profile_system_prompt = """
    You are a highly experienced recruitment director reviewing a candidate's profile.
    You are able to decide what are the most relevant information to evaluate this candidate.
    Please list out the key aspects of the candidate's profile for assessment.
    Here is an example:
    Job title & summary: The candidate's title reflecting seniority and a brief of their current role.
    Key responsibilities: A detailed list of the candidate's daily tasks, expectations, and major contributions.
    Required skills & competencies: the candidate's technical abilities (e.g., software, tools) and behavioural attributes (e.g., leadership, communication).
    Qualifications & experience: the candidate's education, certifications, and previous experience.
    """

    candidate_profile = Website(profile_url)
    profile_user_prompt = f"Here is the candidate's profile:\n"
    profile_user_prompt += candidate_profile.text
    profile_user_prompt += "\nPlease create a succinct and coherent summary of this candidate's profile."
    
    return call_LLM(
        chosen_model=chosen_model,
        sys_prompt= profile_system_prompt,
        user_prompt= profile_user_prompt
    )

In [ ]:
# Set system_prompt
system_prompt = """
You are a highly experienced headhunter.
You will create a short summary about a candidate's fit against a job profile.
You will first score the candidate's overall fit against the job profile between 0 and 100.
100 indicates perfect fit and 0 indicates totally incompatible.
If the person is significantly under-qualified or over-qualified, this should result in a low score.
Please then provide a short summary in the following format:
Overall review outcome: a short summary of your review opinions of the candidate's fit,
Job profile summary: a short summary of the job profile,
Candidate profile summary: a short summary of the candidate profile.
Respond in markdown without code blocks.
"""

In [ ]:
# Define get_user_prompt function with both contents and links
def get_user_prompt(job, profile):
    user_prompt = f"You are looking at a candidate's profile."
    user_prompt += "\nThe contents of the candidate's profile is as follows:\n"
    user_prompt += profile
    user_prompt += f"\nPlease assess the candidate against this job profile."
    user_prompt += "\nThe details of the job profile is as follows:\n"
    user_prompt += job

    user_prompt += "\n Please use all the information to come up with a score \
        of overall fit and a short summary of strengths and areas to probe \
        in markdown without code blocks."

    return user_prompt

In [ ]:
# Define review_summary function
def review_summary(job_url, profile_url, chosen_model):
    # Summarise the job profile using the chosen model
    summarised_job = job_profile_summariser(
        chosen_model=chosen_model,
        job_url=job_url
    )
    # Summarise the candidate profile using the chosen model
    summarised_profile = candidate_profile_summariser(
        chosen_model=chosen_model,
        profile_url=profile_url
    )
    # Generates the prompt for the review summary
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content" : get_user_prompt(
            job = summarised_job,
            profile= summarised_profile
        )}
    ]
    # Generates review summary using the chosen model
    stream = ollama.chat.completions.create(
        model=chosen_model,
        messages=messages,
        stream=True
    )
    # Stream the results
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
job_profile_link = gr.Textbox(label="Job Profile Link:",
                                info="Copy and paste a valid job profile link",
                                lines=2)

candidate_profile_link = gr.Textbox(label="Candidate Profile Link:",
                                info="Copy and paste a valid candidate profile link. \
                                    NOTE: LINKEDIN DOESN'T WORK.",
                                lines=2)

model_selector = gr.Dropdown(OL_MODEL_LIST, label="Select a model")

review_outcome = gr.Markdown(label="Review Outcome:")

view = gr.Interface(
    fn=review_summary,
    title="Profile Reviewer",
    inputs=[job_profile_link, candidate_profile_link, model_selector],
    outputs=[review_outcome],
    examples=[
        [
            "https://www.linkedin.com/jobs/view/4354447639",
            "https://www.hsbc.com/who-we-are/our-people/board-of-directors/pam-kaur",
            OL_MODEL1
        ],
        [
            "https://www.linkedin.com/jobs/view/4352041193",
            "https://edwarddonner.com/about-me-and-about-nebula/",
            OL_MODEL2
        ]
    ],
    flagging_mode="never"
    )
view.launch(inbrowser=True)